In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv("./WA_Fn-UseC_-Telco-Customer-Churn.csv")
df.head(3)

In [ ]:
df.loc[df['PhoneService'] == 'No', 'PhoneService'] = 0
df.loc[df['PhoneService'] == 'Yes', 'PhoneService'] = 1
df['PhoneService']

In [ ]:
df.TotalCharges.fillna(0, inplace=True)
df.TotalCharges = df.TotalCharges.astype('float32')
df.TotalCharges.median(), df.TotalCharges.mean(), df.TotalCharges.std()

In [ ]:
df.loc[df['Churn'] == 'No', 'Churn'] = 0
df.loc[df['Churn'] == 'Yes', 'Churn'] = 1
df['Churn']

In [ ]:
for i in ['StreamingMovies', 'StreamingTV', 'TechSupport']:
    df.loc[df[i] == 'No', i] = 0
    df.loc[df[i] == 'No internet service', i] = 0
    df.loc[df[i] == 'Yes', i] = 1
df[['StreamingMovies', 'StreamingTV', 'TechSupport']].head(5)

In [ ]:
df.PhoneService.fillna(0, inplace=True)

In [ ]:
columns = ['gender', 'tenure', 'PhoneService', 'TotalCharges', 
           'StreamingMovies', 'StreamingTV', 'TechSupport', 'Churn']
#сохраним на всякий случай изначальный датасет
df_start = df 


In [ ]:
df = df[columns]
df.head(3)

In [ ]:
from sklearn.model_selection import train_test_split

features = ['gender', 'tenure', 'PhoneService', 'TotalCharges', 'StreamingMovies', 'StreamingTV', 'TechSupport']
target = 'Churn'
#Ваш код здесь
X_train, X_test, y_train, y_test = train_test_split(df[features], df[target], test_size=0.2, random_state=42)

##### соберем pipeline для поля gender 

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline

class ColumnSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in test_columns:
            if col_ not in self.columns:
                X[col_] = 0
        return X[self.columns]

gender = Pipeline([
                ('selector', ColumnSelector(key='gender')),
                ('ohe', OHEEncoder(key='gender'))
            ])

In [ ]:
gender.fit_transform(X_train)

In [ ]:
from sklearn.preprocessing import StandardScaler

tenure =  Pipeline([
                ('selector', NumberSelector(key='tenure')),
                ('standard', StandardScaler())
            ])

В моделях классификации большой (по сравнению с остальными) разброс у одного параметра может заставить алгоритм переоценивать вес
этого самого параметра при обучении в следствии чего переобучается

In [ ]:
TotalCharges =  Pipeline([
                ('selector', NumberSelector(key='TotalCharges')),
                ('standard', StandardScaler())
            ])

Объединение всех "кубиков" очень легко сделать таким образом

In [ ]:
from sklearn.pipeline import FeatureUnion

number_features = Pipeline([
                ('selector', ColumnSelector(key=['PhoneService',
                                                 'StreamingMovies', 'StreamingTV', 
                                                 'TechSupport']))
            ])

In [ ]:
feats = FeatureUnion([('tenure', tenure),
                      ('TotalCharges', TotalCharges),
                      ('continuos_features', number_features),
                      ('gender', gender)])
feature_processing = Pipeline([('feats', feats)])

На этом этапе что мы сделали:
1. написали преобразователь поля gender, который делает OHE кодирование
2. написали преобразователь для поля tenure, который нормирует и центрирует его 
3. повторили п. 2 для поля TotalCharges
3. для всех остальных просто взяли признаки как они есть, без изменений

У нас уже готов наш пайплайн, который преобразовывает признаки. Давайте обучим модель поверх него. В качестве модели возьмем RandomForestClassifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier

pipeline = Pipeline([
    ('features',feats),
    ('classifier', RandomForestClassifier(random_state = 42)),
])

pipeline.fit(X_train, y_train)

In [ ]:
pipeline.predict(X_test)


In [ ]:
from sklearn.metrics import roc_auc_score, log_loss

roc_auc_score(y_test, y_pred), log_loss(y_test, y_pred)

### Сохраним наш пайплайн

In [ ]:
import dill

In [ ]:
with open("model_RF.dill", "wb") as f:
    dill.dump(pipeline, f)